In [6]:
import librosa
import numpy as np
import os
from pydub.generators import WhiteNoise
from pydub import AudioSegment
import soundfile as sf 
import numpy as np


def preprocess_audio(self, filepath, sr=44100):
        try:
            y, sr = librosa.load(filepath)
            audio_segment = AudioSegment(
                y.tobytes(), 
                frame_rate=sr,
                sample_width=y.dtype.itemsize,
                channels=1  # librosa loads audio in mono
            )
            y = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)
            # y = y / 32768.0  # Normalize the array to be in the range [-1, 1]

            # Detect non-silent segments
            non_silent_intervals = librosa.effects.split(y, top_db=20)  # Adjust top_db as needed

            # Find the effective 1 second segment
            effective_duration = 1  # Desired duration in seconds
            effective_samples = int(effective_duration * sr)  # Convert to number of samples

            # Initialize variables to store the effective segment
            effective_segment = np.array([], dtype=y.dtype)

            # Extract non-silent segments until we reach 1 second
            for start, end in non_silent_intervals:
                segment = y[start:end]
                effective_segment = np.concatenate((effective_segment, segment))
                if len(effective_segment) >= effective_samples:
                    effective_segment = effective_segment[:effective_samples]
                    break

            # If the effective segment is still less than 1 second, pad with silence
            if len(effective_segment) < effective_samples:
                padding = np.zeros(effective_samples - len(effective_segment), dtype=y.dtype)
                effective_segment = np.concatenate((effective_segment, padding))

            
            sf.write(filepath, effective_segment, sr)
            return effective_segment

        except Exception as e:
            print(f"Error processing audio: {e}")
            return None

  
def load_audio(   file_path):
    return AudioSegment.from_file(file_path)

def save_augmented(   audio, file_path, suffix):
    try:
        output_file_path = f"{os.path.splitext(file_path)[0]}_{suffix}.wav"
        audio.export(output_file_path, format='wav')
    except Exception as e:
        print(f"Error saving file {output_file_path}: {e}")

def pad_audio(   augmented_audio, original_length):
    if len(augmented_audio) < original_length:
        padding = AudioSegment.silent(duration=original_length - len(augmented_audio))
        return augmented_audio + padding
    return augmented_audio[:original_length]

def increase_volume(   song, original_length, file_path):
    try:
        louder_song =  pad_audio(song + 50, original_length)
        save_augmented(louder_song, file_path, "louder")
    except Exception as e:
        print(f"Error increasing volume for {file_path}: {e}")

def decrease_volume(   song, original_length, file_path):
    try:
        quieter_song =  pad_audio(song - 20, original_length)
        save_augmented(quieter_song, file_path, "quieter")
    except Exception as e:
        print(f"Error decreasing volume for {file_path}: {e}")

def change_speed_up(   song, original_length, file_path):
    try:
        speed_up =  pad_audio(song.speedup(playback_speed=2), original_length)
        save_augmented(speed_up, file_path, "faster")
    except Exception as e:
        print(f"Error changing speed up for {file_path}: {e}")

def change_speed_down(   song, original_length, file_path):
    try:
        slow_down =  pad_audio(song.speedup(playback_speed=0.6), original_length)
        save_augmented(slow_down, file_path, "slower")
    except Exception as e:
        print(f"Error changing speed down for {file_path}: {e}")

def add_noise(   song, original_length, file_path):
    try:
        noise = WhiteNoise().to_audio_segment(duration=original_length)
        noisy_song =  pad_audio(song.overlay(noise - 30), original_length)
        save_augmented(noisy_song, file_path, "noisy")
    except Exception as e:
        print(f"Error adding noise to {file_path}: {e}")

def fade_in_out(   song, original_length, file_path):
    try:
        fade_in_out_song =  pad_audio(song.fade_in(2000).fade_out(2000), original_length)
        save_augmented(fade_in_out_song, file_path, "fade_in_out")
    except Exception as e:
        print(f"Error adding fade in/out to {file_path}: {e}")

def process_file(   file_path):
    try:
        song =  load_audio(file_path)
        original_length = len(song)

        increase_volume(song, original_length, file_path)
        decrease_volume(song, original_length, file_path)
        change_speed_up(song, original_length, file_path)
        change_speed_down(song, original_length, file_path)
        add_noise(song, original_length, file_path)
        fade_in_out(song, original_length, file_path)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

process_file("2_jump_0.wav")